In [38]:
from nltk.tokenize import word_tokenize
import json
import numpy as np

In [2]:
arquivo_original = open("DBpediaRelations-PT-0.2.txt", encoding="utf-8")

In [3]:
texto_original = arquivo_original.readlines()

In [4]:
texto_original = "\n".join(texto_original)

In [5]:
texto_original = texto_original.replace("****************************\n\nSENTENCE","SENTENCE")

In [6]:
chuncks = texto_original.split("****************************\n")

In [7]:
for s in chuncks[0].split('\n'):
    if len(s)>0: print(s)

SENTENCE : A América Latina localiza-se totalmente no hemisfério ocidental, sendo atravessada pelo Trópico de Câncer, que corta a parte central do México; pelo Equador, que passa pelo Brasil, Colômbia, Equador e toca o norte do Peru; e pelo Trópico de Capricórnio, que atravessa o Brasil, o Paraguai, a Argentina e o Chile.
MANUALLY CHECKED : TRUE
ENTITY1 : América Latina
TYPE1 : LOCATION
ENTITY2 : Argentina
TYPE2 : LOCATION
REL TYPE : locatedInArea


In [21]:
def getEntityStartEnd(entity_name, tokens):
    entity = {}
    indices = []
    for token1 in word_tokenize(entity_name):
        for i in range(len(tokens)):
            token2 = tokens[i]
            if token1 == token2: indices.append(i)
    if len(indices) == 0:
        return None
    novos_indices = []
    novos_indices.append(indices[0])
    for i in range(len(indices)-1):
        if indices[i+1] == indices[i]+1:
            novos_indices.append(indices[i+1])
        else:
            break
    entity['start'] = min(novos_indices)
    entity['end'] = max(novos_indices)+1
    return entity

In [98]:
sentencas = []
for chunck in chuncks:
    sentenca = {'entities':[], 'relations':[]}
    erro = False
    for linha in chunck.split('\n'):
        if len(linha)>0 and not erro:
            if linha.startswith("SENTENCE : "): sentenca['tokens'] = word_tokenize(linha.replace("SENTENCE : ",""))
            if linha.startswith("MANUALLY CHECKED : "): sentenca['manual'] = linha.replace("MANUALLY CHECKED : ","") == 'TRUE'
            if linha.startswith("ENTITY1 : "):
                entity = getEntityStartEnd(entity_name = linha.replace("ENTITY1 : ",""),tokens= sentenca['tokens'])
                erro = entity == None
                sentenca['entities'].append(entity)
            if linha.startswith("ENTITY2 : "):
                entity = getEntityStartEnd(entity_name = linha.replace("ENTITY2 : ",""),tokens= sentenca['tokens'])
                erro = entity == None
                sentenca['entities'].append(entity)
            if linha.startswith("TYPE1 : "):
                sentenca['entities'][0]['type'] = linha.replace("TYPE1 : ","").strip()
            if linha.startswith("TYPE2 : "):
                sentenca['entities'][1]['type'] = linha.replace("TYPE2 : ","").strip()
            if linha.startswith("REL TYPE : "):
                relation = linha.replace("REL TYPE : ","").strip()
                sentenca['relations'].append({'type':relation, 'head':0, 'tail':1})
    if (not erro)\
        and (len(sentenca['relations']) > 0)\
        and ('type' in sentenca['entities'][0])\
        and ('type' in sentenca['entities'][1]):
        sentencas.append(sentenca)
print(len(sentencas))

94274


In [99]:
with open("relextport_train_dev.json", "w", encoding="utf8") as arq:
    json.dump(sentencas, arq, ensure_ascii=False)

In [100]:
np.random.seed(12345)
array_sentencas = np.arange(len(sentencas))
np.random.shuffle(array_sentencas)
array_sentencas

array([41832,  1785, 73671, ...,  2177, 77285, 86498])

In [101]:
qnt_sentencas = len(sentencas)
idx_treino = array_sentencas[:(int(qnt_sentencas*0.7))]

In [102]:
sentencas_treino = []
sentencas_teste = []
for i in range(len(sentencas)):
    if i in idx_treino:
        sentencas_treino.append(sentencas[i])
    else:
        sentencas_teste.append(sentencas[i])
print(len(sentencas_treino))
print(len(sentencas_teste))
    

65991
28283


In [103]:
with open("relextport_train.json", "w", encoding="utf8") as arq:
    json.dump(sentencas_treino, arq, ensure_ascii=False)
with open("relextport_dev.json", "w", encoding="utf8") as arq:
    json.dump(sentencas_teste, arq, ensure_ascii=False)

In [104]:
relations = set()
for s in sentencas:
    if len(s['relations']) > 0:
        relations.add(s['relations'][0]['type'])
    else:
        print(s)
print(relations)

{'parent', 'deathOrBurialPlace', 'influencedBy', 'other', 'partOf', 'origin', 'partner', 'keyPerson', 'locatedInArea', 'successor'}


In [106]:
entities = set()
for s in sentencas:
    if (len(s['entities']) > 0) and ('type' in s['entities'][0].keys()) and ('type' in s['entities'][1].keys()) :
        entities.add(s['entities'][0]['type'])
        entities.add(s['entities'][1]['type'])
    else:
        print(s)
print(entities)

{'LOCATION', 'PERSON', 'ORGANIZATION'}
